# Concentrated Long Only Fund Analysis

In [10]:
import os
import pandas as pd

# Set the relative path to the 'Historical_Data_13F' folder
directory = './Historical_Data_13F_Updated/concentrated_long_only_funds'

# List all CSV files in the 'Historical_Data_13F' directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Check if we have any CSV files to process
if csv_files:
    # Build the full file path for the first CSV file
    first_csv_path = os.path.join(directory, csv_files[0])

    # Load the first CSV file into a DataFrame
    df = pd.read_csv(first_csv_path)

    # Print the first few rows of the DataFrame
    print(df.head())
else:
    print("No CSV files found in the directory.")


                             Stock Symbol  Type  Shares Held  Market Value  \
0                  Microsoft corp.   MSFT   NaN   12189646.0  1.694726e+09   
1              Paypal holdings inc   PYPL   NaN   15402344.0  1.595529e+09   
2      Estee lauder companies inc.     EL   NaN    6766747.0  1.346244e+09   
3                    Stryker corp.    SYK   NaN    5945105.0  1.285926e+09   
4  Philip morris international inc     PM   NaN   16928583.0  1.285387e+09   

   % of Portfolio  Previous % of Portfolio  Ranking  Change in shares  \
0          8.6626                   8.4958      1.0           18512.0   
1          8.1556                   9.1795      2.0           11316.0   
2          6.8814                   6.3553      3.0          105962.0   
3          6.5731                   6.2146      4.0          143665.0   
4          6.5703                   6.2478      5.0         1660076.0   

    % Change Change Type  % Ownership Qtr first owned                  sector  \
0   0.15209

In [11]:
# Path to the directory containing CSV files
directory = './Historical_Data_13F_Updated/concentrated_long_only_funds'

# Initialize an empty dictionary to store the ticker presence data
ticker_presence = {}

# Get all CSV files from the directory
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Iterate over the files and extract unique tickers
for csv_file in csv_files:
    # Strip the '.csv' extension for use as DataFrame column names
    clean_file_name = csv_file.replace('.csv', '')

    file_path = os.path.join(directory, csv_file)
    df = pd.read_csv(file_path)
    unique_tickers = df['Symbol'].unique()
    
    # Update the ticker presence for each file
    for ticker in unique_tickers:
        if ticker not in ticker_presence:
            # Initialize a new entry in the dictionary with the cleaned file names
            ticker_presence[ticker] = {clean_name: 0 for clean_name in [name.replace('.csv', '') for name in csv_files]}
        # Mark the presence of the ticker in the current file
        ticker_presence[ticker][clean_file_name] = 1

# Create a DataFrame from the ticker presence dictionary
one_hot_encoded_df = pd.DataFrame.from_dict(ticker_presence, orient='index')

# Reset the index to get tickers as a column instead of an index
one_hot_encoded_df.reset_index(inplace=True)
one_hot_encoded_df.rename(columns={'index': 'Ticker'}, inplace=True)

# Save the one_hot_encoded_df DataFrame to a CSV file
one_hot_encoded_df.to_csv('concentrated_long_only_fund_encoded_data.csv', index=False)

# Print the first few rows of the one-hot encoded DataFrame to verify
print(one_hot_encoded_df.head())

  Ticker  fundsmith-2019_q3  fundsmith-2021_q4  fundsmith-2019_q4  \
0   MSFT                  1                  1                  1   
1   PYPL                  1                  1                  1   
2     EL                  1                  1                  1   
3    SYK                  1                  1                  1   
4     PM                  1                  1                  1   

   fundsmith-2021_q1  fundsmith-2021_q3  fundsmith-2021_q2  fundsmith-2020_q4  \
0                  1                  1                  1                  1   
1                  1                  1                  1                  1   
2                  1                  1                  1                  1   
3                  1                  1                  1                  1   
4                  1                  1                  1                  1   

   fundsmith-2020_q2  fundsmith-2020_q3  fundsmith-2020_q1  
0                  1                 

## Isolating Tickers to Pull Updated Historical Data

This being a dynamic analysis with updating information each quarter, the holdings within the funds and the historical pricing data of each financial instrument changes. Because of this, we are going to pull a list of all the unique individual tickers and run them through whatever data provider, scraper, or API can give us historical pricing. Because I do not have access to a tool like Bloomberg for example, I am going to pull them from [Yahoo Finance](http://localhost:8888/lab/tree/Documents/Github/capital-markets/Yahoo%20Finance%20API%20Data%20Pull.ipynb), which you can access using the provided link.

Once doing so, we are going to update that information, place them into the Historical Data folder, which we will then access, clean and create a new dataframe inside of this project to access.

In [19]:

# Extract the 'Ticker' column into a new DataFrame
tickers_df = pd.DataFrame(one_hot_encoded_df['Ticker'])

# Display the first few rows to verify
print(tickers_df.head(60))

# If you want to save this to a new CSV file:
#tickers_df.to_csv('list_of_tickers.csv', index=False)

   Ticker
0    MSFT
1    PYPL
2      EL
3     SYK
4      PM
5    INTU
6    IDXX
7      FB
8     WAT
9     MKC
10    PEP
11      V
12    ADP
13    BDX
14    JNJ
15   BF.B
16    EFX
17   MASI
18   VRSK
19   SABR
20   CHKP
21   ANSS
22   CGNX
23   IPGP
24   VRSN
25   MSCI
26    AOS
27   PAYC
28    MAR
29    CHD
30   MELI
31    MMM
32    CDK
33    NKE
34   AMZN
35   SBUX
36  GOOGL
37   FTNT
38   QLYS
39   WING
40    ROL
41     HD
42    ZTS
43     PG
44    TSM
45    WNS
46      G
47     CL
48     XP
49    CLX


## Exploratory Data Analysis

In [12]:
df = one_hot_encoded_df

In [13]:

# Add a new column that sums how many quarters each company is held
df['Quarters_Held'] = df.iloc[:, 1:].sum(axis=1)

# Sort the DataFrame by the 'Quarters_Held' column in descending order
df_sorted = df.sort_values(by='Quarters_Held', ascending=False)

# Extract just the 'Ticker' and 'Quarters_Held' columns for a clear ranking
ranked_companies = df_sorted[['Ticker', 'Quarters_Held']]

# Display or save the ranked companies list
print(ranked_companies)


   Ticker  Quarters_Held
0    MSFT             10
15   BF.B             10
30   MELI             10
27   PAYC             10
26    AOS             10
1    PYPL             10
24   VRSN             10
23   IPGP             10
22   CGNX             10
21   ANSS             10
19   SABR             10
18   VRSK             10
17   MASI             10
16    EFX             10
25   MSCI             10
14    JNJ             10
2      EL             10
12    ADP             10
11      V             10
10    PEP             10
9     MKC             10
8     WAT             10
7      FB             10
6    IDXX             10
5    INTU             10
4      PM             10
3     SYK             10
13    BDX              9
28    MAR              9
29    CHD              7
48     XP              7
20   CHKP              7
33    NKE              7
35   SBUX              7
38   QLYS              6
47     CL              5
37   FTNT              5
44    TSM              4
43     PG              4


## Uploading Historical Pricing Data

In [20]:
MSFT  = pd.read_csv('Historical_Data_Prices_Cleaned/MSFT.csv')
PYPL = pd.read_csv('Historical_Data_Prices_Cleaned/PYPL.csv')
EL = pd.read_csv('Historical_Data_Prices_Cleaned/EL.csv')
SYK = pd.read_csv('Historical_Data_Prices_Cleaned/SYK.csv')
PM = pd.read_csv('Historical_Data_Prices_Cleaned/PM.csv')
INTU = pd.read_csv('Historical_Data_Prices_Cleaned/INTU.csv')
IDXX = pd.read_csv('Historical_Data_Prices_Cleaned/IDXX.csv')
FB = pd.read_csv('Historical_Data_Prices_Cleaned/FB.csv')
WAT = pd.read_csv('Historical_Data_Prices_Cleaned/WAT.csv')
MKC = pd.read_csv('Historical_Data_Prices_Cleaned/MKC.csv')
PEP = pd.read_csv('Historical_Data_Prices_Cleaned/PEP.csv')
V = pd.read_csv('Historical_Data_Prices_Cleaned/V.csv')
ADP = pd.read_csv('Historical_Data_Prices_Cleaned/ADP.csv')
BDX = pd.read_csv('Historical_Data_Prices_Cleaned/BDX.csv')
JNJ = pd.read_csv('Historical_Data_Prices_Cleaned/JNJ.csv')
BF.B = pd.read_csv('Historical_Data_Prices_Cleaned/BF.B.csv')
EFX = pd.read_csv('Historical_Data_Prices_Cleaned/EFX.csv')
MASI = pd.read_csv('Historical_Data_Prices_Cleaned/MASI.csv')
VRSK = pd.read_csv('Historical_Data_Prices_Cleaned/VRSK.csv')
SABR = pd.read_csv('Historical_Data_Prices_Cleaned/SABR.csv')
CHKP = pd.read_csv('Historical_Data_Prices_Cleaned/CHKP.csv')
ANSS = pd.read_csv('Historical_Data_Prices_Cleaned/ANSS.csv')
CGNX = pd.read_csv('Historical_Data_Prices_Cleaned/CGNX.csv')
IPGP = pd.read_csv('Historical_Data_Prices_Cleaned/IPGP.csv')
VRSN = pd.read_csv('Historical_Data_Prices_Cleaned/VRSN.csv')
MSCI = pd.read_csv('Historical_Data_Prices_Cleaned/MSCI.csv')
AOS = pd.read_csv('Historical_Data_Prices_Cleaned/AOS.csv')
PAYC = pd.read_csv('Historical_Data_Prices_Cleaned/PAYC.csv')
MAR = pd.read_csv('Historical_Data_Prices_Cleaned/MAR.csv')
CHD = pd.read_csv('Historical_Data_Prices_Cleaned/CHD.csv')
MELI = pd.read_csv('Historical_Data_Prices_Cleaned/MELI.csv')
MMM = pd.read_csv('Historical_Data_Prices_Cleaned/MMM.csv')
CDK = pd.read_csv('Historical_Data_Prices_Cleaned/CDK.csv')
NKE = pd.read_csv('Historical_Data_Prices_Cleaned/NKE.csv')
AMZN = pd.read_csv('Historical_Data_Prices_Cleaned/AMZN.csv')
SBUX = pd.read_csv('Historical_Data_Prices_Cleaned/SBUX.csv')
GOOGL = pd.read_csv('Historical_Data_Prices_Cleaned/GOOGL.csv')
FTNT = pd.read_csv('Historical_Data_Prices_Cleaned/FTNT.csv')
QLYS = pd.read_csv('Historical_Data_Prices_Cleaned/QLYS.csv')
WING = pd.read_csv('Historical_Data_Prices_Cleaned/WING.csv')
ROL = pd.read_csv('Historical_Data_Prices_Cleaned/ROL.csv')
HD = pd.read_csv('Historical_Data_Prices_Cleaned/HD.csv')
ZTS = pd.read_csv('Historical_Data_Prices_Cleaned/ZTS.csv')
PG = pd.read_csv('Historical_Data_Prices_Cleaned/PG.csv')
TSM = pd.read_csv('Historical_Data_Prices_Cleaned/TSM.csv')
WNS = pd.read_csv('Historical_Data_Prices_Cleaned/WNS.csv')
G = pd.read_csv('Historical_Data_Prices_Cleaned/G.csv')
CL = pd.read_csv('Historical_Data_Prices_Cleaned/CL.csv')
XP = pd.read_csv('Historical_Data_Prices_Cleaned/XP.csv')
CLX = pd.read_csv('Historical_Data_Prices_Cleaned/CLX.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Historical_Data_Prices_Cleaned/PYPL.csv'